## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2019-04-27 15:59:27--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1, 2620:100:6026:1::a27d:4601
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/dl/obaitrix9jyu84r/quora.txt [переход]
--2019-04-27 15:59:27--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc610006d63f8892580aa372095f.dl.dropboxusercontent.com/cd/0/get/Afy-2HbtNORVaGYD4gDf06iMYQZLcyiBWmCqY8w61J8_GnleirkmC9ObfeMrOSySM35ZNRBG1zJbbZOIB5ICkIa_S1zLqLsuvXs-9GtWcZrw_Q/file?dl=1# [переход]
--2019-04-27 15:59:27--  https://uc610006d63f8892580aa372095f.dl.dropboxusercontent.com/cd/0/get/Afy-2HbtNORVaGYD4gDf06iMYQZLcyiBWmCqY8w61J8_GnleirkmC9ObfeMrOSySM35ZNRBG1zJbbZOIB5ICkIa_S1zLqLsuvXs-9GtWcZrw_Q/file?dl=1
Распозна

In [1]:
import numpy as np

data = list(open("./quora.txt"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [2]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [3]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(data[i].lower()) for i in range(len(data))]

In [4]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [5]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [6]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

/home/keleas/anaconda3/envs/keras/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [7]:
# now you can get word vectors !
model.get_vector('anything')

array([-0.20235762,  0.70160383,  1.1871333 , -0.7545285 , -1.5617903 ,
       -3.0902498 , -0.7908679 , -6.202348  ,  3.5254803 ,  0.4914876 ,
       -0.01842561, -0.2605386 , -1.2323282 ,  1.019187  , -1.9455829 ,
        2.045476  ,  1.4654524 , -0.39259923,  0.22513545, -2.9841788 ,
       -2.4862525 ,  0.2731452 , -2.5179913 ,  0.69746476, -3.4577575 ,
        0.38095924, -1.0220178 ,  1.5847924 ,  0.7473814 , -2.1506171 ,
        1.7094114 ,  1.8431375 ], dtype=float32)

In [8]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9538142681121826),
 ('cheese', 0.9286336898803711),
 ('sauce', 0.9272717237472534),
 ('butter', 0.9263729453086853),
 ('potato', 0.9254540205001831),
 ('soup', 0.9175244569778442),
 ('flour', 0.9167293310165405),
 ('beans', 0.9136219024658203),
 ('chicken', 0.911468505859375),
 ('honey', 0.9109708070755005)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [9]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [10]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155143737793),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843312203884125),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [11]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [12]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])

In [13]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [14]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2)
pca.fit(word_vectors)
word_vectors_pca = pca.transform(word_vectors)
word_vectors_pca = scale(normalize(word_vectors_pca, axis=0), axis=0)

/home/keleas/anaconda3/envs/keras/lib/python3.7/site-packages/sklearn/preprocessing/data.py:197: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [15]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [16]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [17]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1003', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [18]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca


word_tsne = TSNE().fit_transform(word_vectors)
word_tsne = scale(normalize(word_tsne, axis=0), axis=0)

/home/keleas/anaconda3/envs/keras/lib/python3.7/site-packages/sklearn/preprocessing/data.py:197: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [19]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1110', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [22]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
#     data_tok = tokenizer.tokenize(phrase.lower())
#     vectors = []
#     for tok in data_tok:
#         if tok in model.vocab.keys():
#             vectors.append(np.array(model.get_vector(tok)))
#     vectors = np.array(vectors)
    
    vector = []
    for word in tokenizer.tokenize(phrase.lower()):
        if word in model.vocab:
            vector.append(model.get_vector(word))
    if len(vector) == 0:
        return np.zeros([model.vector_size], dtype='float32')
    
    return np.mean(vector, axis=0)
        
    

In [24]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")
assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [26]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]
# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [27]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [25]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(verbose=1000).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1001 samples in 0.003s...
[t-SNE] Computed neighbors for 1001 samples in 0.224s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1001
[t-SNE] Computed conditional probabilities for sample 1001 / 1001
[t-SNE] Mean sigma: 0.477874
[t-SNE] Computed conditional probabilities in 0.044s
[t-SNE] Iteration 50: error = 80.4878998, gradient norm = 0.2956020 (50 iterations in 6.030s)
[t-SNE] Iteration 100: error = 80.8781662, gradient norm = 0.2888527 (50 iterations in 6.532s)
[t-SNE] Iteration 150: error = 82.3905182, gradient norm = 0.2845711 (50 iterations in 7.242s)
[t-SNE] Iteration 200: error = 81.4741287, gradient norm = 0.3002517 (50 iterations in 7.294s)
[t-SNE] Iteration 250: error = 82.7633286, gradient norm = 0.2878391 (50 iterations in 7.554s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 82.763329
[t-SNE] Iteration 300: error = 2.1351771, gradient norm = 0.0035603 (50 iterations in 5.749s)

In [26]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1226', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [30]:
data_vectors = [get_phrase_embedding(l) for l in data]

In [92]:
from sklearn.metrics.pairwise import cosine_distances
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    
    query_vector = get_phrase_embedding(query)
    distance = cosine_distances(data_vectors, query_vector[None, :])
    return [data[i] for i in np.argsort(distance[:, 0])[:k]]

In [93]:
find_nearest("Hello, my name is")

['What is my first name, middle name, and last name?\n',
 'If my full name is Gowtham Palanisamy, then what is my first name, middle name and last name?\n',
 'My name on my id is K Lakshmi Narayana. So what is my first name, middle name and last name?\n',
 'My name in PAN is Rekapalli Naga Sai Lakshmi Bharani. What is my First name, middle name and last name and Rekapalli is my surname?\n',
 'If my name is K. Vamsi Krishna, then what are my first name and last name?\n',
 'What are first, middle and last names? My name is Senthil Nathan and my father name is Subramaniam. In my certificate it is S.Senthilnathan.\n',
 'What is my name?\n',
 'If my name is KANNAN A what is my first, middle and last name?\n',
 'My Chinese Name is Chenlingya, I want to change my English name.Lydia, Nicole, Lesley and Claire, but which one is the best?I like traveling,singing\n',
 "What is the meaning of my name, 'Nitanshu'?\n"]

In [94]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [95]:
find_nearest(query="How does Trump?", k=10)

['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [96]:
find_nearest(query="Why don't i ask a question myself?", k=10)

["Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "I don't have no question?\n",
 "Why can't I take a joke?\n",
 "Why don't I ever get a girl?\n",
 "Can I ask a girl out that I don't know?\n",
 "Why don't I have a girlfriend?\n"]

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.